In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from os import listdir, path
import numpy as np
import random
from collections import defaultdict
import datetime
                                  
random.seed(42)
seq_length  = 36
num_classes = 10 # 0 to 9
dim = 6 # accX,accY,accZ,gyrX,gyrY,gyrZ


In [2]:
def padding(data):
    """Get neighboor padding."""
    padded_data = []
    noise_level = [ 20, 20, 20, 0.2, 0.2, 0.2 ]
    
    # Before- Neighbour padding
    tmp_data = (np.random.rand(seq_length, dim) - 0.5) * noise_level + data[0]
    tmp_data[(seq_length -
              min(len(data), seq_length)):] = data[:min(len(data), seq_length)]
    padded_data.append(tmp_data)
    # After- Neighbour padding
    tmp_data = (np.random.rand(seq_length, dim) - 0.5) * noise_level + data[-1]
    tmp_data[:min(len(data), seq_length)] = data[:min(len(data), seq_length)]
    padded_data.append(tmp_data)
    return padded_data

In [3]:
def build_dataset(data, label):
    """Support function for format.(Helps format train, valid and test.)"""
    # Add 2 padding, initialize data and label
    padded_num = 2
    length = len(data) * padded_num
    features = np.zeros((length, seq_length, dim))
    labels = np.zeros(length)
    # Get padding for train, valid and test
    for idx, (data, label) in enumerate(zip(data, label)):
        padded_data = padding(data)
        for num in range(padded_num):
            features[padded_num * idx + num] = padded_data[num]
            labels[padded_num * idx + num] = label
    # Turn into tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((features, labels.astype("int32")))
    return length, dataset

In [4]:
def time_warping(molecule, denominator, data):
  """Generate (molecule/denominator)x speed data."""
  tmp_data = [[0
               for i in range(len(data[0]))]
              for j in range((int(len(data) / molecule) - 1) * denominator)]
  for i in range(int(len(data) / molecule) - 1):
    for j in range(len(data[i])):
      for k in range(denominator):
        tmp_data[denominator * i +
                 k][j] = (data[molecule * i + k][j] * (denominator - k) +
                          data[molecule * i + k + 1][j] * k) / denominator
  return tmp_data


In [5]:
def augment_data(original_data, original_label):
  """Perform data augmentation."""
  new_data = []
  new_label = []
  for idx, (data, label) in enumerate(zip(original_data, original_label)):  # pylint: disable=unused-variable
    # Original data
    new_data.append(data)
    new_label.append(label)
    # Sequence shift
    for num in range(5):  # pylint: disable=unused-variable
      new_data.append((np.array(data, dtype=np.float32) +
                       (random.random() - 0.5) * 200).tolist())
      new_label.append(label)
    # Random noise
    tmp_data = [[0 for i in range(len(data[0]))] for j in range(len(data))]
    for num in range(5):
      for i in range(len(tmp_data)):
        for j in range(len(tmp_data[i])):
          tmp_data[i][j] = data[i][j] + 5 * random.random()
      new_data.append(tmp_data)
      new_label.append(label)
    # Time warping
    fractions = [(3, 2), (5, 3), (2, 3), (3, 4), (9, 5), (6, 5), (4, 5)]
    for molecule, denominator in fractions:
      new_data.append(time_warping(molecule, denominator, data))
      new_label.append(label)
    # Movement amplification
    for molecule, denominator in fractions:
      new_data.append(
          (np.array(data, dtype=np.float32) * molecule / denominator).tolist())
      new_label.append(label)
  return new_data, new_label

In [6]:
def load_data(data_type, files):
    data   = []
    labels = []
    random.shuffle(files)
   
    for file in files:
        with open(file) as f:
            label = path.splitext(file)[0][-1]
            labels.append(label)
            readings = []
            for line in f:
                reading = line.strip().split(',')
                readings.append([float(i) for i in reading[0:6]])

            data.append(readings)
            
    if data_type == 'train':
        data, labels = augment_data(data, labels)
    
    return build_dataset(data, labels)

In [7]:
files_path = defaultdict(list)
dir = './data'
for filename in listdir(dir):
    if filename.endswith('.csv'): #or filename.endswith('1.csv')  or filename.endswith('2.csv')  or filename.endswith('3.csv')  or filename.endswith('4.csv'):
        digit = path.splitext(filename)[0][-1]
        files_path[digit].append(path.join(dir, filename))

train_files      = []
validation_files = []
test_files       = []

for digit in files_path:
    random.shuffle(files_path[digit])
    
    train_split = int(len(files_path[digit]) * 0.6) # 60%
    validation_split = train_split + int(len(files_path[digit]) * 0.2) # 20%

    train_files += files_path[digit][:train_split]
    validation_files += files_path[digit][train_split:validation_split]
    # remaining 20%
    test_files += files_path[digit][validation_split:]

train_length, train_data = load_data('train', train_files)
validation_length, validation_data = load_data('validation', validation_files)
test_length, test_data = load_data('test', test_files )

print(train_length, validation_length, test_length)

for (ds, lb) in test_data.take(1):
    print(ds)

22200 290 312
tf.Tensor(
[[ 8.62218550e+02  2.14691843e+02  7.33347481e+02 -6.70498390e+00
  -8.70778775e+00 -2.16604876e+01]
 [ 8.52860236e+02  2.23494644e+02  7.39960521e+02 -6.56941050e+00
  -8.72773746e+00 -2.15468501e+01]
 [ 8.66868736e+02  2.09782966e+02  7.29041796e+02 -6.59991350e+00
  -8.68658239e+00 -2.16797709e+01]
 [ 8.68028236e+02  2.25161774e+02  7.25009907e+02 -6.58862671e+00
  -8.80797946e+00 -2.16260231e+01]
 [ 8.51485842e+02  2.23154272e+02  7.29396976e+02 -6.63546198e+00
  -8.79622833e+00 -2.15226978e+01]
 [ 8.50965643e+02  2.10079153e+02  7.24618299e+02 -6.67961745e+00
  -8.62681530e+00 -2.15870825e+01]
 [ 8.53347917e+02  2.26176597e+02  7.38116982e+02 -6.51793153e+00
  -8.68970041e+00 -2.16693844e+01]
 [ 8.53488090e+02  2.17294139e+02  7.29503386e+02 -6.58085326e+00
  -8.71020467e+00 -2.16735914e+01]
 [ 8.63223763e+02  2.21538970e+02  7.39704110e+02 -6.67661895e+00
  -8.76696546e+00 -2.16423769e+01]
 [ 8.54412862e+02  2.06821531e+02  7.27370296e+02 -6.51437605e+00


In [8]:
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu", input_shape=(seq_length, dim, 1)),
      tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
      tf.keras.layers.MaxPool2D((2, 2)),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
      tf.keras.layers.MaxPool2D((2, 2), padding="same"),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation="relu"),
      tf.keras.layers.MaxPool2D((2, 2), padding="same"),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation="relu"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(num_classes, activation="softmax")
  ])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 36, 6, 8)          80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 6, 8)          584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 18, 3, 8)          0         
_________________________________________________________________
dropout (Dropout)            (None, 18, 3, 8)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 3, 8)          584       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 2, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 2, 8)           0

In [10]:
epochs = 100
batch_size = 64
steps_per_epoch=1000

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

def reshape_function(data, label):
  reshaped_data = tf.reshape(data, [-1, dim, 1])
  return reshaped_data, label

train_data = train_data.map(reshape_function)
validation_data = validation_data.map(reshape_function)

train_data = train_data.batch(batch_size).repeat()
validation_data = validation_data.batch(batch_size)

logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# %load_ext tensorboard
# %tensorboard --logdir logdir

model.fit(
  train_data,
  epochs=epochs,
  validation_data=validation_data,
  steps_per_epoch=steps_per_epoch,
  validation_steps=int((validation_length - 1) / batch_size + 1),
  callbacks=[tensorboard_callback])

Train for 1000 steps, validate for 5 steps
Epoch 1/100
1000/1000 [==============================] - 4s 4ms/step - loss: 2.9807 - accuracy: 0.1402 - val_loss: 2.1285 - val_accuracy: 0.1828
Epoch 2/100
1000/1000 [==============================] - 3s 3ms/step - loss: 2.0924 - accuracy: 0.1999 - val_loss: 1.8769 - val_accuracy: 0.2862
Epoch 3/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8780 - accuracy: 0.3077 - val_loss: 1.6219 - val_accuracy: 0.4414
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.6085 - accuracy: 0.4414 - val_loss: 1.3866 - val_accuracy: 0.5931
Epoch 5/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.3297 - accuracy: 0.5515 - val_loss: 1.0012 - val_accuracy: 0.6931
Epoch 6/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1468 - accuracy: 0.6195 - val_loss: 0.8652 - val_accuracy: 0.7552
Epoch 7/100
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0249 - a

Epoch 57/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2096 - accuracy: 0.9330 - val_loss: 0.2791 - val_accuracy: 0.9414
Epoch 58/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2075 - accuracy: 0.9332 - val_loss: 0.2939 - val_accuracy: 0.9345
Epoch 59/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2072 - accuracy: 0.9322 - val_loss: 0.2664 - val_accuracy: 0.9552
Epoch 60/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2082 - accuracy: 0.9334 - val_loss: 0.4275 - val_accuracy: 0.9276
Epoch 61/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2077 - accuracy: 0.9327 - val_loss: 0.3687 - val_accuracy: 0.9448
Epoch 62/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2000 - accuracy: 0.9370 - val_loss: 0.2534 - val_accuracy: 0.9483
Epoch 63/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.2039 - accuracy: 0.9340 - val_loss: 0.2295 -

In [11]:
test_data = test_data.map(reshape_function)
test_labels = np.zeros(test_length)

idx = 0
for data, label in test_data:
    test_labels[idx] = label.numpy()
    idx += 1
    
test_data = test_data.batch(batch_size)

loss, acc = model.evaluate(test_data)
pred = np.argmax(model.predict(test_data), axis=1)
confusion = tf.math.confusion_matrix(labels=tf.constant(test_labels), predictions=tf.constant(pred), num_classes=num_classes)
print(confusion)


5/5 [==============================] - 0s 2ms/step - loss: 0.2605 - accuracy: 0.9327
tf.Tensor(
[[34  0  0  0  0  0  6  0  0  0]
 [ 0 36  0  0  0  0  0  0  0  0]
 [ 0  0 39  0  0  0  1  0  0  0]
 [ 0  0  0 40  0  0  0  0  0  0]
 [ 0  0  0  0 44  0  0  0  0  0]
 [ 0  0  0  0  0 40  0  0  0  0]
 [ 4  0  0  0  0  0 38  0  0  0]
 [ 1  0  1  0  0  0  0  7  0  1]
 [ 0  0  1  0  0  0  1  0  8  0]
 [ 0  0  0  0  0  0  2  3  0  5]], shape=(10, 10), dtype=int32)


In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("model_quantized.tflite", "wb").write(tflite_model)


21544